<a href="https://colab.research.google.com/github/AbhinavPudasaini/hackathon/blob/main/Loan_Credit_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import optuna
import xgboost as xgb
import shap


# These dataset are from loan default credit scoring datat science competiton

In [ ]:
datat = pd.read_csv('/content/Train1.csv')
datae = pd.read_csv('/content/Test (1).csv')

In [ ]:
datat.info()

In [ ]:
datat.describe()

Down in the code when i ched the feature importance using SHAP, these columns were the most ireelevant for the model training and formation.

In [ ]:
datat.drop(['duration', 'lender_id', 'New_versus_Repeat','ID', 'customer_id', 'country_id','tbl_loan_id'],axis=1,inplace=True)
datae.drop(['duration', 'lender_id', 'New_versus_Repeat','ID', 'customer_id', 'country_id','tbl_loan_id'],axis=1,inplace=True)
datat.drop(['disbursement_date','due_date'],axis=1, inplace = True)
datae.drop(['disbursement_date','due_date'],axis=1, inplace = True)

I also tried creating different columns, like ration between total and total amount to repay so make good prediction.

In [ ]:
datat['total_amount_ratio'] = datat['Total_Amount_to_Repay'] / datat['Total_Amount']
datae['total_amount_ratio'] = datae['Total_Amount_to_Repay'] / datae['Total_Amount']
datat['Other_amount_for_loan'] = datat['Total_Amount_to_Repay'] - datat['Lender_portion_to_be_repaid']
datae['Other_amount_for_loan'] = datae['Total_Amount_to_Repay'] - datae['Lender_portion_to_be_repaid']
datat['left_amount_for_loan'] = datat['Total_Amount'] - datat['Amount_Funded_By_Lender']
datae['left_amount_for_loan'] = datae['Total_Amount'] - datae['Amount_Funded_By_Lender']

I used mainly catboost so i didn't needed to transform categorical variable like this cause catboost can directly encode them. But catboost mainly use one-hot-encoding which increase dimension unnecessarily so i used labelencoder.In tree base algorithms using label encoder in categorical varibale will not affect.

In [ ]:
le = LabelEncoder()
custom_order = ['Type_1', 'Type_2', 'Type_3', 'Type_4','Type_5','Type_6','Type_7','Type_8','Type_9','Type_10','Type_11','Type_12','Type_13','Type_14','Type_15','Type_16','Type_17','Type_18','Type_19','Type_20','Type_21','Type_22','Type_23', 'Type_24']  # Desired order
le = LabelEncoder()
le.fit(custom_order)
# le.fit(datat['loan_type'])
datat['loan_type'] = le.transform(datat['loan_type'])
datae['loan_type'] = le.transform(datae['loan_type'])

In [ ]:
X_train = datat.drop(['target'],axis=1)
y_train = datat['target']
X_test = datae

Since the data and mainly target variable is highly imbalance, I computed the class weight and then tried balancing to get better results.

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(datat['target']), y=datat['target'])

X_train, X_temp, y_train, y_temp = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)


I used optuna to get the best hyperparameter.

In [ ]:

def objective(trial):
    iterations = trial.suggest_int("iterations", 1000, 8000)
    depth = trial.suggest_int("depth", 4, 10)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 2, 8)
    colsample_bylevel = trial.suggest_float("colsample_bylevel", 0.5, 1.0)
    class_weights = [0.5093329 , 25.18696343]

    # Initializing the CatBoost model with the suggested hyperparameters
    model = CatBoostClassifier(
        iterations=iterations,
        depth=depth,
        learning_rate=learning_rate,
        l2_leaf_reg=l2_leaf_reg,
        colsample_bylevel=colsample_bylevel,
        eval_metric='F1',
        random_seed=42,
        class_weights=class_weights,
        verbose=0
    )

    model.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=20, verbose=0)

    y_pred = model.predict(X_valid)

    f1 = f1_score(y_valid, y_pred)
    return f1


In [ ]:
study = optuna.create_study(direction="maximize")

# Start optimization
# You can adjust the number of trials
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)


In [ ]:

# Example weights for a binary classification
#This is the result i got from above compute class weight function
class_weights = [0.5093329 , 27.18696343]

model = CatBoostClassifier(
    iterations=40,
    learning_rate=0.106077355116654,
    depth=10,
    class_weights=class_weights,
    eval_metric='F1',
    random_seed=42,
    l2_leaf_reg = 8.171923654908364,
    colsample_bylevel =0.5733702387068401,
    early_stopping_rounds=42
)

model.fit(X_train, y_train,eval_set=(X_valid, y_valid))

In [ ]:
y_test = model.predict(datae)
sample = pd.read_csv("/content/SampleSubmission.csv")
new_data = pd.DataFrame({'ID': sample['ID'], 'target': y_test})
new_data.to_csv('submission_new_03.csv', index=False)

## Below i have done experiment with different models too like xgboost, randomforest, svm ,logistic regression. Svm actually didn't run or we can say that it took long long time because the dataset contains almost 70000 datas.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.1, random_state=42)


# I also did some experiments with Xgboost::

In [ ]:
import xgboost as xgb
from sklearn.utils.class_weight import compute_sample_weight

# Example weights for a binary classification
class_weights = [0.5093329, 27.18696343]  # Adjust based on dataset imbalance

# Compute sample weights for each instance in the training set based on class weights
# sample_weights = compute_sample_weight(class_weight={0: class_weights[0], 1: class_weights[1]}, y=y_train)

model_xgb = xgb.XGBClassifier(
    n_estimators=5000,
    learning_rate=0.05,
    max_depth=6,
    subsample = 0.7,
    scale_pos_weight=class_weights[1] / class_weights[0],  # Adjust class imbalance scaling
    eval_metric='logloss',  # For binary classification, you can also use 'logloss' or 'error'
    random_state=42,
    reg_lambda=4,  # L2 regularization (Ridge)
    reg_alpha=0.1 # L1 regularization (Lasso)    colsample_bytree=0.83,
)

model_xgb.fit(X_train, y_train, verbose=100)


I used SHAP to get the best feature and also improve score.

In [ ]:
import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_valid)


In [ ]:
# Summary plot for all SHAP values
shap.summary_plot(shap_values, X_valid)


In [ ]:
shap.summary_plot(shap_values, X_valid, plot_type="bar")


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(class_weight=class_weights)
model.fit(X_train, y_train)


In [ ]:
import pandas as pd

# Define custom binning logic
def loan_type_bins(value):
    if value in ["Type_14",
"Type_2"	,
"Type_11"	,
"Type_18"	,
"Type_17"	,
"Type_12"	,
"Type_23"	,
"Type_20"	,
"Type_16"	,
"Type_13"	,
"Type_19"	,
"Type_15"	,
"Type_21"	,
"Type_24"	,
"Type_22"]:  # Example group
        return 'Type_1_100'
    elif value in ["Type_10",	"Type_6"	,"Type_9"	]:  # Another group
        return 'Type_100_500'
    else:
        return value

# Apply the function to create a new column
datat['loan_type_bins'] = datat['loan_type'].apply(loan_type_bins)
datae['loan_type_bins'] = datae['loan_type'].apply(loan_type_bins)


# Preview data
# print(df[['loan_type', 'loan_type_bin']].head())


In [ ]:
data = pd.get_dummies(datat['loan_type']).astype(int)
dataee = pd.get_dummies(datae['loan_type']).astype(int)

In [ ]:
# datat = pd.concat([datat, data], axis=1)
# datae = pd.concat([datae, dataee], axis=1)

In [ ]:
param_grid = {
    'iterations': [1000, 1250, 1500],   # Number of trees
    'learning_rate': [0.01, 0.05, 0.1],  # Step size for updating weights
    'depth': [6, 8, 10],              # Depth of each tree
    'l2_leaf_reg': [1, 3, 5],        # L2 regularization
}

In [ ]:
model = CatBoostClassifier(
    cat_features=['loan_type','New_versus_Repeat'],
    loss_function='Logloss',     # For binary classification
    eval_metric='Accuracy',      # Evaluation metric
    verbose=False                # Suppress output during training
)


In [ ]:
grid_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=10,
    scoring='accuracy',         # Adjust based on your metric
    cv=3,                       # 3-fold cross-validation
    verbose=1,
    n_jobs=-1

)

# Fit Grid Search
grid_search.fit(X_train, y_train)

# Print best parameters
print("Best Parameters:", grid_search.best_params_)
# Best Parameters: {'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 1000, 'depth': 6, 'border_count': 64}

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

model = LogisticRegression()
model.fit(X_train, y_train)
# Best Parameters: {'learning_rate': 0.05, 'l2_leaf_reg': 1, 'iterations': 1000, 'depth': 6, 'border_count': 64}


In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.svm import SVC

svm = SVC(kernel="rbf", gamma=0.5, C=1.0)
# Trained the model
svm.fit(X_train, y_train)


In [ ]:
new_df = datat[datat['target'] == 1]
new_df1 = datat.sample(35000)
df = pd.concat([new_df, new_df1], axis=0, ignore_index=True)
X_train = datat.drop(['target'],axis=1)
y_train = datat['target']

X_test = datae

In [ ]:
# Initialize CatBoostClassifier
from catboost import CatBoostClassifier
cat_features=['loan_type','New_versus_Repeat'], l2_leaf_reg = 4,
model = CatBoostClassifier(iterations=3000, learning_rate=0.05, depth=6,colsample_bylevel = 0.8 ,early_stopping_rounds=1300)

# Fit the model
model.fit(X_train, y_train)


In [ ]:
from sklearn.preprocessing import LabelEncoder
lee = LabelEncoder()

custom_order = ["Type_1",
"Type_2"	,
"Type_3"	,
"Type_4"	,
"Type_5"	,
"Type_6"	,
"Type_7"	,
"Type_8"	,
"Type_9"	,
"Type_10"	,
"Type_11"	,
"Type_12"	,
"Type_13"	,
"Type_14"	,
"Type_15"	,
"Type_16"	,
"Type_17"	,
"Type_18",
"Type_19",
"Type_20",
"Type_21",
"Type_22",
"Type_23",
"Type_24"]

lee.fit(custom_order)
datat['loan_type'] = lee.transform(datat['loan_type'])

datae['loan_type'] = lee.transform(datae['loan_type'])


In [ ]:
X_train = datat.drop(['target'],axis=1)
y_train = datat['target']
X_test = datae

In [ ]:
import xgboost as xgb


# model = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False, eval_metric='logloss')
# model.fit(X_train, y_train)

# # Make predictions
# y_pry_tested = model.predict(X_test)

# Convert to XGBoost's DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)  # Test data doesn't need labels


In [ ]:
params = {
    'objective': 'binary:logistic',   # For binary classification
    'max_depth': 25,
    'learning_rate': 0.07,
    'n_estimators': 500,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'reg_lambda': 1,
    'reg_alpha': 0.5,
    'eval_metric': 'logloss'
}


In [ ]:
num_boost_round = 500
model = xgb.train(params, dtrain, num_boost_round)


In [ ]:
y_pred_prob = model.predict(dtest)
y_pred = [1 if prob > 0.45 else 0 for prob in y_pred_prob]